## Classifying From Tabular Data

In [ ]:
# To classify tabular data we use fastai.tabular
from fastai.tabular.all import * 
import matplotlib.pyplot as plt
import numpy
import random

## Reading Data

In [ ]:
# Path to data set
# Test data get partitioned into a separate file already now
from pathlib import Path
data_path = Path('./data/mecs/MECS_2-Phase-Steels.csv')

In [ ]:
# Reading CSV file containing training and validation data
import pandas as pandas
dataframe = pandas.read_csv(data_path, sep=';')

# Looking at the first data
dataframe.head()

## Data Loading and Encoding 

In [ ]:
batch_size = 4
n_features = len(dataframe.columns)-1

def extract_numpy_from_df( dataframe, y_column ):
    
    # please extend your source code here
    
    return X,Y

def clean_data( X ):

    # please extend your source code here
    
    return X
    
def get_validation_and_training_indices( dataset_length ):

    # please extend your source code here
    
    return indices_train,indices_validation

def normalize( X ):

    # please extend your source code here
    
    return X

def hot_1_encode( Y, codes ):
    
    # please extend your source code here
    
    return Y_encoded

def create_batch( permutation, batch_no, batch_size, X ):
    x_batch = torch.zeros( [ batch_size, X.shape[1] ], dtype=torch.float32 )
    
    # please extend your source code here
    
    return x_batch

codes = { 'category_1' : 0, 'category_2' : 1, 'category_3' : 2 }

X,Y = extract_numpy_from_df( dataframe, 0 )
X   = clean_data( X )
indices_train,indices_validation = get_validation_and_training_indices( X.shape[0] )

X   = normalize( X )
Y   = hot_1_encode( Y, codes )

## Creating a Network Architecture

In [ ]:
def create_model( n_in, n_hidden, n_out ):
    layer = []

    # please extend your source code here

    model = nn.Sequential( *layer )
    return model

model = create_model(X.shape[1], [50, 25], Y.shape[1])

In [ ]:
print(model)

## Training Loop

In [ ]:
def accuracy_metric( y, y_hat ):
    y     = torch.argmax(y, dim=1).to(torch.float32)
    y_hat = torch.argmax(y_hat, dim=1).to(torch.float32)
    difference = y_hat-y
    return 1.0 - torch.mean( torch.abs( difference ) ).item() 

def train_one_epoch( epoch_index, indices_train, X, Y, optimizer, loss_fn, batch_size ):
    losses           = []
    accuracies       = []
    
    # please extend your source code here
    
    return losses, accuracies

optimizer = torch.optim.Adam( params = model.parameters(), lr=0.001 )
loss_fn   = torch.nn.CrossEntropyLoss()

losses           = []
accuracies       = []
n_epochs = 50
for i in range (n_epochs):
    metrics = train_one_epoch( i, indices_train, X, Y, optimizer, loss_fn, 32 )
    loss_per_epoch, accuracy_per_epoch = metrics
    losses = losses + loss_per_epoch
    accuracies = accuracies + accuracy_per_epoch
    print("epoch",i,"/",n_epochs,"accuracy",accuracy_per_epoch[-1])

In [ ]:
fig,axis = plt.subplots( 1,1, figsize=(16,8) )
axis.plot( losses )
axis.plot( accuracies )
plt.show()

## Hyper Parameter Tuning